In [ ]:
import pandas as pd
import numpy as np

# ----------------------
# 1. 基础参数配置（可手动调整）
# ----------------------
target_group_id = 3  # 目标组编号
initial_capital = 1000000  # 初始资金：100万
history_data_path = r"D:\workspace\xiaoyao\high_corr_stock_groups_2005_2024.csv"  # 历史分组
price_all_path = r"D:\workspace\xiaoyao\data\stock_daily_price.parquet"  # 2005-2025全量数据
deviation_threshold = 0.02  # 回归卖出阈值：偏离组内平均价≤2%
history_quantile = 0.05  # 买入阈值：低于历史5%分位数（显著低估）
# 明确2025年时间范围（避免跨年数据）
start_date_2025 = pd.Timestamp("2025-01-01")
end_date_2025 = pd.Timestamp("2025-12-31")

# ----------------------
# 2. 数据加载与预处理（严格拆分年份）
# ----------------------
# 2.1 加载历史分组，提取目标组的股票代码
group_df = pd.read_csv(history_data_path)
target_group_row = group_df[group_df["组号"] == target_group_id]
if target_group_row.empty:
    raise ValueError(f"未找到组编号{target_group_id}，请检查历史分组文件中的组号是否正确")
target_stocks = target_group_row["组内股票代码"].iloc[0].split(",")
print(f"目标组{target_group_id}的股票列表：{target_stocks}")

# 2.2 加载全量价格数据，严格拆分“历史数据（2005-2024）”和“2025年回测数据”
price_all = pd.read_parquet(price_all_path)
# 确保date列为datetime格式（避免字符串格式导致筛选失效）
price_all["date"] = pd.to_datetime(price_all["date"])

# 拆分1：2025年回测数据（仅用当年数据）
price_2025 = price_all[
    (price_all["date"] >= start_date_2025) & 
    (price_all["date"] <= end_date_2025)
].copy()
# 转为“行=日期，列=股票代码”的收盘价矩阵（2025年）
close_2025 = price_2025.pivot_table(
    index="date", 
    columns="stock_code", 
    values="close"
)
# 筛选目标组股票，删除2025年数据缺失过多的股票（保留≥80%交易日数据）
close_2025_target = close_2025[target_stocks].dropna(
    thresh=int(len(close_2025)*0.8),  # 至少80%交易日有数据
    axis=1
)
valid_stocks = close_2025_target.columns.tolist()
if len(valid_stocks) < 3:
    raise ValueError(f"目标组{target_group_id}在2025年有效股票不足3只，无法进行回测")
print(f"目标组{target_group_id}在2025年有有效数据的股票：{valid_stocks}（共{len(valid_stocks)}只）")

# 拆分2：历史数据（2005-2024，仅用于计算买入阈值，不参与回测）
price_hist = price_all[price_all["date"] < start_date_2025].copy()  # 严格小于2025年
# 转为“行=日期，列=股票代码”的收盘价矩阵（历史）
close_hist = price_hist.pivot_table(
    index="date", 
    columns="stock_code", 
    values="close"
)
# 筛选目标组的有效股票（与2025年一致，确保阈值对应）
close_hist_target = close_hist[valid_stocks].dropna(
    thresh=int(len(close_hist)*0.5),  # 历史数据至少50%交易日有数据
    axis=1
)
if len(close_hist_target.columns) < len(valid_stocks)*0.5:
    raise ValueError(f"目标组{target_group_id}的历史数据（2005-2024）缺失过多，无法计算买入阈值")

# 2.3 计算历史偏离度阈值（仅用2005-2024年数据）
# 历史组内平均价
hist_benchmark = close_hist_target.mean(axis=1)
# 历史偏离度：（股价-平均价）/平均价（标准化，消除股价绝对值影响）
hist_deviation = (close_hist_target - hist_benchmark.values.reshape(-1, 1)) / hist_benchmark.values.reshape(-1, 1)
# 每个股票的历史5%分位数（买入阈值：低于此值为显著低估）
buy_deviation_threshold = hist_deviation.quantile(history_quantile, axis=0)
print(f"\n目标组股票的买入偏离度阈值（2005-2024年5%分位数）：\n{buy_deviation_threshold.round(4)}")

# ----------------------
# 3. 全仓轮动套利模拟（仅遍历2025年交易日）
# ----------------------
# 初始化变量
current_capital = initial_capital
position = None  # 持仓记录：{"股票代码": "", "买入价": "", "买入日期": "", "股数": ""}
trade_log = []  # 交易日志
daily_nav = []  # 每日净资产（用于资金曲线）

# 遍历2025年的每个交易日（按日期升序，仅2025年）
for date in sorted(close_2025_target.index):
    # 当日目标组股票收盘价（排除当日无数据的）
    daily_close = close_2025_target.loc[date].dropna()
    # 当日有效股票不足50%，跳过（避免极端情况）
    if len(daily_close) < len(valid_stocks)*0.5:
        daily_nav.append({"日期": date, "净资产": current_capital})
        continue
    
    # 计算当日组内平均价（基准）
    daily_benchmark = daily_close.mean()
    # 计算当日每只股票的偏离度：（股价-平均价）/平均价（负=低于平均价）
    daily_deviation = (daily_close - daily_benchmark) / daily_benchmark
    # 筛选符合条件的股票：1.低于平均价；2.低于历史买入阈值
    eligible_stocks = daily_deviation[
        (daily_deviation < 0) &  # 低于组内平均价
        (daily_deviation <= buy_deviation_threshold[daily_deviation.index])  # 显著低估
    ]
    
    # ----------------------
    # 步骤1：处理卖出（持仓股票回归平均价）
    # ----------------------
    if position is not None:
        stock = position["股票代码"]
        if stock in daily_close.index:  # 当日有该股票数据
            current_price = daily_close[stock]
            current_dev = (current_price - daily_benchmark) / daily_benchmark
            # 卖出条件：偏离度回归到±2%（接近正常水平）
            if abs(current_dev) <= deviation_threshold:
                # 计算卖出收益（扣除0.1%手续费）
                sell_amount = position["股数"] * current_price
                fee = sell_amount * 0.001  # 卖出手续费
                profit = sell_amount - fee - (position["买入价"] * position["股数"])
                current_capital += profit
                
                # 记录交易日志（日期已锁定2025年）
                trade_log.append({
                    "日期": date.strftime("%Y-%m-%d"),
                    "操作": "卖出",
                    "股票代码": stock,
                    "买入价": round(position["买入价"], 2),
                    "卖出价": round(current_price, 2),
                    "股数": position["股数"],
                    "持仓天数": (date - position["买入日期"]).days,
                    "收益金额": round(profit, 2),
                    "收益比例": round(profit/(position["买入价"]*position["股数"])*100, 2),
                    "卖出后净资产": round(current_capital, 2)
                })
                
                # 清空持仓
                position = None
                print(f"【{date.strftime('%Y-%m-%d')}】卖出 {stock}，收益{round(profit,2)}元，净资产{round(current_capital,2)}元")
    
    # ----------------------
    # 步骤2：处理买入（无持仓+有符合条件的低估股票）
    # ----------------------
    if position is None and not eligible_stocks.empty:
        # 选“低于平均价最多”的股票（偏离度最小）
        target_stock = eligible_stocks.idxmin()
        target_price = daily_close[target_stock]
        target_dev = eligible_stocks.min()
        
        # 全仓买入（扣除0.1%手续费）
        buy_amount = current_capital * (1 - 0.001)  # 可买金额（扣手续费）
        buy_shares = int(buy_amount / target_price)  # 可买股数（取整）
        actual_buy_cost = buy_shares * target_price
        
        # 记录持仓
        position = {
            "股票代码": target_stock,
            "买入价": target_price,
            "买入日期": date,
            "股数": buy_shares
        }
        
        # 扣减资金
        current_capital -= actual_buy_cost
        
        # 记录交易日志
        trade_log.append({
            "日期": date.strftime("%Y-%m-%d"),
            "操作": "买入",
            "股票代码": target_stock,
            "买入价": round(target_price, 2),
            "股数": buy_shares,
            "买入成本": round(actual_buy_cost, 2),
            "买入后净资产": round(current_capital, 2),
            "买入时偏离度": round(target_dev*100, 2)  # 负百分比：低于平均价的比例
        })
        
        print(f"【{date.strftime('%Y-%m-%d')}】买入 {target_stock}，价格{round(target_price,2)}元，股数{buy_shares}股，净资产{round(current_capital,2)}元")
    
    # 记录当日净资产（含持仓市值）
    if position is not None and position["股票代码"] in daily_close.index:
        position_value = position["股数"] * daily_close[position["股票代码"]]
        daily_total_nav = current_capital + position_value
    else:
        daily_total_nav = current_capital
    daily_nav.append({
        "日期": date.strftime("%Y-%m-%d"),
        "净资产": round(daily_total_nav, 2)
    })

# ----------------------
# 4. 回测结果统计与输出（验证2025年数据）
# ----------------------
# 处理未平仓持仓（按2025年最后一个交易日收盘价计算）
final_date_2025 = sorted(close_2025_target.index)[-1]
if position is not None and position["股票代码"] in close_2025_target.loc[final_date_2025].index:
    final_price = close_2025_target.loc[final_date_2025, position["股票代码"]]
    final_position_value = position["股数"] * final_price
    final_capital = current_capital + final_position_value
    trade_log.append({
        "日期": final_date_2025.strftime("%Y-%m-%d"),
        "操作": "回测结束未平仓",
        "股票代码": position["股票代码"],
        "持仓市值": round(final_position_value, 2),
        "最终净资产": round(final_capital, 2)
    })
else:
    final_capital = current_capital

# 计算核心指标（仅基于2025年数据）
total_return = (final_capital - initial_capital) / initial_capital * 100
trading_days_2025 = len(daily_nav)  # 实际回测的2025年交易日数
annual_return = total_return * (250 / trading_days_2025) if trading_days_2025 > 0 else 0
# 计算最大回撤
daily_nav_df = pd.DataFrame(daily_nav)
daily_nav_df["累计最大净资产"] = daily_nav_df["净资产"].cummax()
daily_nav_df["回撤"] = (daily_nav_df["净资产"] - daily_nav_df["累计最大净资产"]) / daily_nav_df["累计最大净资产"] * 100
max_drawdown = daily_nav_df["回撤"].min()

# 输出结果（验证无2005年数据）
print("\n" + "="*80)
print(f"2025年组{target_group_id}全仓偏离度套利回测结果（无历史年份干扰）")
print("="*80)
print(f"回测时间范围：{start_date_2025.strftime('%Y-%m-%d')} 至 {end_date_2025.strftime('%Y-%m-%d')}")
print(f"初始资金：{initial_capital:,.2f}元")
print(f"最终净资产：{final_capital:,.2f}元")
print(f"总收益率：{total_return:.2f}%")
print(f"年化收益率：{annual_return:.2f}%")
print(f"最大回撤：{max_drawdown:.2f}%")
print(f"2025年实际交易日数：{trading_days_2025}天")
print(f"总交易笔数：{len(trade_log)}笔（买入+卖出+未平仓）")
print("="*80)

# 保存结果（文件名含2025年标识，避免混淆）
trade_log_df = pd.DataFrame(trade_log)
daily_nav_df = pd.DataFrame(daily_nav)
trade_log_path = rf"D:\workspace\xiaoyao\group{target_group_id}_2025_arbitrage_trade_log.csv"
daily_nav_path = rf"D:\workspace\xiaoyao\group{target_group_id}_2025_daily_nav.csv"
trade_log_df.to_csv(trade_log_path, index=False, encoding="utf-8-sig")
daily_nav_df.to_csv(daily_nav_path, index=False, encoding="utf-8-sig")
print(f"\n交易日志已保存至：{trade_log_path}")
print(f"每日净资产已保存至：{daily_nav_path}")

目标组3的股票列表：['000552.XSHE', '000933.XSHE', '000937.XSHE', '000983.XSHE', '600123.XSHG', '600188.XSHG', '600348.XSHG', '600395.XSHG', '600508.XSHG', '600971.XSHG', '600997.XSHG', '601001.XSHG', '601666.XSHG', '601699.XSHG', '601898.XSHG']
目标组3在2025年有有效数据的股票：['000552.XSHE', '000933.XSHE', '000937.XSHE', '000983.XSHE', '600123.XSHG', '600188.XSHG', '600348.XSHG', '600395.XSHG', '600508.XSHG', '600971.XSHG', '600997.XSHG', '601001.XSHG', '601666.XSHG', '601699.XSHG', '601898.XSHG']（共15只）
目标组股票的买入偏离度阈值（历史5%分位数）：
stock_code
000552.XSHE   -0.740470
000933.XSHE    0.490248
000937.XSHE   -0.239681
000983.XSHE    0.021869
600123.XSHG    0.266430
600188.XSHG   -0.383935
600348.XSHG   -0.046149
600395.XSHG   -0.662223
600508.XSHG   -0.427131
600971.XSHG   -0.004486
600997.XSHG   -0.616741
601001.XSHG   -0.705919
601666.XSHG   -0.680514
601699.XSHG   -0.115451
601898.XSHG   -0.878920
Name: 0.05, dtype: float64
【2005-02-01 00:00:00】买入 600348.XSHG，价格12.0元，股数83250股，净资产1000.0元
【2005-09-21 00:00:00】卖出 600

C:\Users\jay\AppData\Local\Temp\ipykernel_38716\3731029196.py:113: RuntimeWarning: invalid value encountered in double_scalars
  "收益比例": round(profit/(position["买入价"]*position["股数"])*100, 2),
C:\Users\jay\AppData\Local\Temp\ipykernel_38716\3731029196.py:113: RuntimeWarning: invalid value encountered in double_scalars
  "收益比例": round(profit/(position["买入价"]*position["股数"])*100, 2),
C:\Users\jay\AppData\Local\Temp\ipykernel_38716\3731029196.py:113: RuntimeWarning: invalid value encountered in double_scalars
  "收益比例": round(profit/(position["买入价"]*position["股数"])*100, 2),
C:\Users\jay\AppData\Local\Temp\ipykernel_38716\3731029196.py:113: RuntimeWarning: invalid value encountered in double_scalars
  "收益比例": round(profit/(position["买入价"]*position["股数"])*100, 2),
C:\Users\jay\AppData\Local\Temp\ipykernel_38716\3731029196.py:113: RuntimeWarning: invalid value encountered in double_scalars
  "收益比例": round(profit/(position["买入价"]*position["股数"])*100, 2),
C:\Users\jay\AppData\Local\Temp\ipykerne

【2007-03-22 00:00:00】卖出 600348.XSHG，收益0.0元，净资产-5.65元
【2007-03-22 00:00:00】买入 601699.XSHG，价格20.56元，股数0股，净资产-5.65元
【2007-06-22 00:00:00】卖出 601699.XSHG，收益0.0元，净资产-5.65元
【2007-06-22 00:00:00】买入 600395.XSHG，价格12.3元，股数0股，净资产-5.65元
【2008-10-07 00:00:00】卖出 600395.XSHG，收益0.0元，净资产-5.65元
【2008-10-07 00:00:00】买入 601699.XSHG，价格25.82元，股数0股，净资产-5.65元
【2009-01-09 00:00:00】卖出 601699.XSHG，收益0.0元，净资产-5.65元
【2009-03-02 00:00:00】买入 601699.XSHG，价格27.26元，股数0股，净资产-5.65元
【2009-03-20 00:00:00】卖出 601699.XSHG，收益0.0元，净资产-5.65元
【2009-04-13 00:00:00】买入 000552.XSHE，价格13.4元，股数0股，净资产-5.65元


C:\Users\jay\AppData\Local\Temp\ipykernel_38716\3731029196.py:113: RuntimeWarning: invalid value encountered in double_scalars
  "收益比例": round(profit/(position["买入价"]*position["股数"])*100, 2),
C:\Users\jay\AppData\Local\Temp\ipykernel_38716\3731029196.py:113: RuntimeWarning: invalid value encountered in double_scalars
  "收益比例": round(profit/(position["买入价"]*position["股数"])*100, 2),
C:\Users\jay\AppData\Local\Temp\ipykernel_38716\3731029196.py:113: RuntimeWarning: invalid value encountered in double_scalars
  "收益比例": round(profit/(position["买入价"]*position["股数"])*100, 2),
C:\Users\jay\AppData\Local\Temp\ipykernel_38716\3731029196.py:113: RuntimeWarning: invalid value encountered in double_scalars
  "收益比例": round(profit/(position["买入价"]*position["股数"])*100, 2),



2025年组3全仓偏离度套利回测结果
初始资金：1,000,000.00元
最终净资产：-5.65元
总收益率：-100.00%
年化收益率：-132.28%
最大回撤：-106.16%
总交易笔数：54笔（买入+卖出）
2025年交易日数：189天

交易日志已保存至：group3_arbitrage_trade_log_2025.csv
每日净资产已保存至：group3_daily_nav_2025.csv
